In [0]:
# Databricks notebook source
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, FloatType, StructType, StructField
from pyspark.sql import SQLContext, Row
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql.functions import lit
from pyspark.sql.functions import unix_timestamp, from_unixtime
import re
import phonenumbers 
import datetime
import time

In [0]:
# User schema to format date type like account_id, cash_balance, bonus_balance which  converts string value to Float value

user_schema = StructType([
            StructField("unique_account_id", StringType(), False),
            StructField("account_id", IntegerType(), True),
            StructField("frontend_id", StringType(), True),
            StructField("network_id", StringType(), True),
            StructField("frontend_name", StringType(), True),
            StructField("parent_frontend", StringType(), True),
            StructField("alias", StringType(), True),
            StructField("first_name", StringType(), True),
            StructField("last_name", StringType(), True),
            StructField("email", StringType(), True),
            StructField("gender", StringType(), True),
            StructField("city", StringType(), True),
            StructField("state", StringType(), True),
            StructField("country", StringType(), True),
            StructField("zip_code", StringType(), True),
            StructField("address", StringType(), True),
            StructField("phone", StringType(), True),
            StructField("subscribed", StringType(), True),
            StructField("bonusable", StringType(), True),
            StructField("card_validated", StringType(), True),
            StructField("depositor_status", StringType(), True),
            StructField("account_status", StringType(), True),
            StructField("currency", StringType(), True),
            StructField("bank", FloatType(), True),
            StructField("winnings", FloatType(), True),
            StructField("cash_balance", FloatType(), True),
            StructField("bonus_balance", FloatType(), True),
            StructField("loyalty", FloatType(), True),
            StructField("net_cash", FloatType(), True),
            StructField("date_of_birth", StringType(), True),
            StructField("registration_date", StringType(), True),
            StructField("registration_code", StringType(), True),
            StructField("acquisition_date", StringType(), True),
            StructField("last_login_date", StringType(), True),
            StructField("affiliate_id", StringType(), True),
            StructField("tracker_id", StringType(), True),
            StructField("referral_link", StringType(), True)])

In [0]:
# Function to clean User Data 
def cleanUserRow(c):
    # get the Mobile field
    number = c.phone

    # initialize variables 
    clean_number = None
    valid_mail = None
    gender = ""

    p = None

    if number is not None:
        try:
            p = phonenumbers.parse(number, None)
            clean_number = "%s%s" % (p.country_code, p.national_number)            
        except:
            p = None

    # validate Email 
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", c.email):
      valid_mail = c.email
    
    
    # validate Gener
    if (c.gender == "female"): 
      gender = "F"
    elif (c.gender =="male"):
      gender = "M"
      
    # replace special characters in name
    firstName = re.sub(r"[^a-zA-Z0-9]+", ' ',  str(c.first_name))
    lastName = re.sub(r"[^a-zA-Z0-9]+", ' ', str(c.last_name))
    city =   re.sub(r"[^a-zA-Z0-9]+", ' ', str(c.city))
    state =   re.sub(r"[^a-zA-Z0-9]+", ' ', str(c.state))
    return Row( 
		unique_account_id = c.unique_account_id,
        account_id = c.account_id,
        frontend_id = c.frontend_id,
        network_id = c.network_id,
        frontend_name = c.frontend_name,
        parent_frontend = c.parent_frontend,
        alias = c.alias,
        first_name = firstName,
        last_name = lastName,
        email = valid_mail,
        gender= gender,
        city = city,
        state = state,
        country= c.country ,
        zip_code = c.zip_code,
        address = c.address,
        phone =  clean_number,
        subscribed = c.subscribed,
        bonusable = c.bonusable,
        card_validated = c.card_validated,
        depositor_status = c.depositor_status,
        account_status = c.account_status,
        currency = c.currency,
        bank = c.bank,
        winnings = c.winnings,
        cash_balance = c.cash_balance,
        bonus_balance = c.bonus_balance,
        loyalty = c.loyalty,
        net_cash = c.net_cash,
        date_of_birth = c.date_of_birth,
        registration_date = c.registration_date,
        registration_code = c.registration_code,
        acquisition_date =  c.acquisition_date,
        last_login_date = c.last_login_date,
        affiliate_id = c.affiliate_id,
        tracker_id = c.tracker_id,
        referral_link = c.referral_link
    )


In [0]:
#We get everyday Users Data file. 
#Clean all files and fields for special char

# We need to locate latest records (old + new) and mark them with timestamps to find right record.
# Let's create a userkey on it as well to define a unique record. The unique id is unique_account_id here
# get data from file
newData = sqlContext.read.format("com.databricks.spark.csv") \
                    .option("header", "true") \
                    .schema (user_schema) \
                    .load("/FileStore/tables/User_Sample-97107.csv")
#get data from table
oldData= sqlContext.read.format("com.databricks.spark.csv") \
                  .option("header","true") \
                  .schema (user_schema) \
                  .load("/FileStore/tables/User_Sample_2-1f270.csv")
# clean the user data (Old & new)
newData_rdd = newData.rdd.map(lambda c: cleanUserRow(c))
newData_df =  sqlContext.createDataFrame(newData_rdd, user_schema)
newData_df = newData_df.fillna('')

# add timestamp and userSeq column
windowSpec = W.orderBy("unique_account_id", "account_id","frontend_id","network_id","frontend_name", "parent_frontend")
timestamp = datetime.datetime.fromtimestamp(time.time())

newData_df = newData_df.withColumn("UserSeq",  F.row_number().over(windowSpec)) \
                       .withColumn("EventTimestamp", lit(timestamp))
# Make sure that columns are in specific order when inserting to hive table 
# Do the casting for Date, Float data's
newData_df = newData_df.select(newData_df.UserSeq.cast(IntegerType()).alias('UserSeq'), \
                               "unique_account_id", "account_id", "frontend_id", "network_id", \
                               "frontend_name", "parent_frontend", "alias", "first_name", "last_name", \
                               "email", "gender", "city", "state", "country", "zip_code", "address", "phone", \
                               "subscribed", "bonusable", "card_validated", "depositor_status", "account_status", \
                               "currency", "bank", "winnings", "cash_balance", "bonus_balance", "loyalty", "net_cash", \
                               from_unixtime(unix_timestamp("date_of_birth", "dd-MMM-yyyy")).alias("date_of_birth"), \
                               from_unixtime(unix_timestamp('registration_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('registration_date'), \
                               "registration_code", 
                               from_unixtime(unix_timestamp('acquisition_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('acquisition_date'), \
                               from_unixtime(unix_timestamp('acquisition_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('last_login_date'), \
                               "affiliate_id", "tracker_id", "referral_link", "EventTimestamp")
  


# Clean the user old data
oldData_rdd = oldData.rdd.map(lambda c: cleanUserRow(c))
oldData_df =  sqlContext.createDataFrame(oldData_rdd, user_schema)
oldData_df = oldData_df.fillna('')
# Add UserSeq and EventTimestamp column
windowSpec = W.orderBy("unique_account_id", "account_id","frontend_id","network_id","frontend_name", "parent_frontend")
oldData_df = oldData_df.withColumn("UserSeq",  F.row_number().over(windowSpec)) \
                       .withColumn("EventTimestamp", lit(timestamp))

# Make sure that columns are in specific order when inserting to hive table 
# Do the casting for Date, Float data's

oldData_df = oldData_df.select(oldData_df.UserSeq.cast(IntegerType()).alias('UserSeq'), \
                               "unique_account_id", "account_id", "frontend_id", "network_id", \
                               "frontend_name", "parent_frontend", "alias", "first_name", "last_name", \
                               "email", "gender", "city", "state", "country", "zip_code", "address", "phone", \
                               "subscribed", "bonusable", "card_validated", "depositor_status", "account_status", \
                               "currency", "bank", "winnings", "cash_balance", "bonus_balance", "loyalty", "net_cash", \
                               from_unixtime(unix_timestamp("date_of_birth", "dd-MMM-yyyy")).alias("date_of_birth"), \
                               from_unixtime(unix_timestamp('registration_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('registration_date'), \
                               "registration_code", 
                               from_unixtime(unix_timestamp('acquisition_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('acquisition_date'), \
                               from_unixtime(unix_timestamp('acquisition_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('last_login_date'), \
                               "affiliate_id", "tracker_id", "referral_link", "EventTimestamp")
  

#.drop("seq") uncomment this code once table is created
newData_df.write.mode("overwrite").saveAsTable("newData")
oldData_df.write.mode("overwrite").saveAsTable("oldData")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-245449772468577> in <module> () 
 5 # Let's create a userkey on it as well to define a unique record. The unique id is unique_account_id here 
 6 # get data from file 
 ----> 7 newData = sqlContext . read . format ( "com.databricks.spark.csv" ) . option ( "header" , "true" ) . schema ( user_schema ) . load ( "/FileStore/tables/User_Sample-97107.csv" ) 
 8 #get data from table 
 9 oldData = sqlContext . read . format ( "com.databricks.spark.csv" ) . option ( "header" , "true" ) . schema ( user_schema ) . load ( "/FileStore/tables/User_Sample_2-1f270.csv" ) 

 NameError : name 'user_schema' is not defined

In [0]:
#Inner join old and new dataframes to get updated records 
resultdf = newData_df.alias("df").join(oldData_df, (newData_df["unique_account_id"] == oldData_df["unique_account_id"])).select("df.*")
#Left Outer Join new and old dataframes to get new records
resultdf2 = newData_df.alias("df").join(oldData_df, (newData_df["unique_account_id"] == oldData_df["unique_account_id"]),"left_outer")
resdf2 = resultdf2.where(F.isnull(oldData_df["unique_account_id"])).select("df.*")
resdf2.count()
#merge all the records 
# why we need resdf1??
# finaldf = resultdf.unionAll(resdf2).unionAll(resdf1)
finaldf = resultdf.unionAll(resdf2)

# Add UserSeq and EventTimestamp column
timestamp = datetime.datetime.fromtimestamp(time.time())
finaldf = finaldf.withColumn("EventTimestamp", lit(timestamp))
windowSpec = W.orderBy("unique_account_id", "account_id","frontend_id","network_id","frontend_name", "parent_frontend")
finaldf = finaldf.withColumn("UserSeq",  F.row_number().over(windowSpec))
# Make sure that columns are in specific order
finaldf = finaldf.select(finaldf.UserSeq.cast(IntegerType()).alias('UserSeq'), \
                               "unique_account_id", finaldf.account_id.cast(IntegerType()).alias('account_id'), "frontend_id", "network_id", \
                               "frontend_name", "parent_frontend", "alias", "first_name", "last_name", \
                               "email", "gender", "city", "state", "country", "zip_code", "address", "phone", \
                               "subscribed", "bonusable", "card_validated", "depositor_status", "account_status", \
                               "currency", \
                               finaldf.bank.cast(FloatType()).alias('bank'),
                               finaldf.winnings.cast(FloatType()).alias('winnings'), \
                               finaldf.cash_balance.cast(FloatType()).alias('cash_balance'), \
                               finaldf.bonus_balance.cast(FloatType()).alias('bonus_balance'), \
                               finaldf.loyalty.cast(FloatType()).alias('loyalty'), \
                               finaldf.net_cash.cast(FloatType()).alias('net_cash'), \
                               from_unixtime(unix_timestamp("date_of_birth", "dd-MMM-yyyy")).alias("date_of_birth"), \
                               from_unixtime(unix_timestamp('registration_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('registration_date'), \
                               "registration_code", \
                               from_unixtime(unix_timestamp('acquisition_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('acquisition_date'), \
                               from_unixtime(unix_timestamp('acquisition_date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MM-yyyy HH:mm:ss').alias('last_login_date'), \
                               "affiliate_id", "tracker_id", "referral_link", "EventTimestamp")

finaldf.write.mode("overwrite").saveAsTable("userData")
 
# Each reocrd must be timestamped to locate the latest date on a record based on unique_account_id

In [0]:
finaldf.printSchema()

root
-- UserSeq: integer (nullable = true)
-- unique_account_id: string (nullable = false)
-- account_id: integer (nullable = true)
-- frontend_id: string (nullable = false)
-- network_id: string (nullable = false)
-- frontend_name: string (nullable = false)
-- parent_frontend: string (nullable = false)
-- alias: string (nullable = false)
-- first_name: string (nullable = false)
-- last_name: string (nullable = false)
-- email: string (nullable = false)
-- gender: string (nullable = false)
-- city: string (nullable = false)
-- state: string (nullable = false)
-- country: string (nullable = false)
-- zip_code: string (nullable = false)
-- address: string (nullable = false)
-- phone: string (nullable = false)
-- subscribed: string (nullable = false)
-- bonusable: string (nullable = false)
-- card_validated: string (nullable = false)
-- depositor_status: string (nullable = false)
-- account_status: string (nullable = false)
-- currency: string (nullable = false)
-- bank: float (nullable = true)
-- winnings: float (nullable = true)
-- cash_balance: float (nullable = true)
-- bonus_balance: float (nullable = true)
-- loyalty: float (nullable = true)
-- net_cash: float (nullable = true)
-- date_of_birth: string (nullable = true)
-- registration_date: string (nullable = true)
-- registration_code: string (nullable = false)
-- acquisition_date: string (nullable = true)
-- last_login_date: string (nullable = true)
-- affiliate_id: string (nullable = false)
-- tracker_id: string (nullable = false)
-- referral_link: string (nullable = false)
-- EventTimestamp: timestamp (nullable = false)

In [0]:
#Lets load differet transaction types in Table with timestamp ,assign numeric keys link to transactions table , drop ID received here and create new one
# Need one more column named "TransactionCategory" and it must be updated with values from  TxType	TxSubType & Source1	Source2	Source3	Source4

userTransactionType= sqlContext.read.format("com.databricks.spark.csv").option("header","true").load("/FileStore/tables/userTransactionTypelatest.csv")
# create transaction_id and EventTimestamp column
userTransactionType.drop("ID")
windowSpec = W.orderBy("TxType","TxSubType","Source1","Source2", "Source3", "Source4")
timestamp = datetime.datetime.fromtimestamp(time.time())
userTransactionType_df = userTransactionType.withColumn("TransactionId",  F.row_number().over(windowSpec)) \
                                            .withColumn("EventTimestamp", lit(timestamp))
# make sure that columns are in specific order while inserting in hive table
userTransactionType_df = userTransactionType_df.select(userTransactionType_df.TransactionId.cast(IntegerType()).alias('TransactionId'), \
                                                       "FinalCategory","TxType", "TxSubType", "Source1", "Source2", "Source3", "Source4" , "EventTimestamp")
userTransactionType_df.show()
# Create the UserTransactionType table from Dataframe.
userTransactionType_df.write.mode("overwrite").saveAsTable("userTransactionType")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-245449772468776> in <module> () 
 5 # create transaction_id and EventTimestamp column 
 6 userTransactionType . drop ( "ID" ) 
 ----> 7 windowSpec = W . orderBy ( "TxType" , "TxSubType" , "Source1" , "Source2" , "Source3" , "Source4" ) 
 8 timestamp = datetime . datetime . fromtimestamp ( time . time ( ) ) 
 9 userTransactionType_df = userTransactionType . withColumn ( "TransactionId" , F . row_number ( ) . over ( windowSpec ) ) . withColumn ( "EventTimestamp" , lit ( timestamp ) ) 

 NameError : name 'W' is not defined

In [0]:
transactions= sqlContext.read.format("com.databricks.spark.csv").option("header","true") .load("/FileStore/tables/User_Trans-0bac9.csv")
#Lets load differet transaction types in Table and assign numeric keys to them to link to Transaction table , drop ID received here  
transaction_df = transactions.drop("transaction_id")
join_df = transaction_df.join (userTransactionType_df, \
                               (userTransactionType_df.TxType == transaction_df.transaction_type) & \
                               (userTransactionType_df.TxSubType == transaction_df.sub_transaction_type) & \
                               (userTransactionType_df.Source1 == transaction_df.source1) & \
                               (userTransactionType_df.Source2 == transaction_df.source2) & \
                               (userTransactionType_df.Source3 == transaction_df.source3) & \
                               (userTransactionType_df.Source4 == transaction_df.source4))

# add seq and Timestamp column
timestamp = datetime.datetime.fromtimestamp(time.time())

windowSpec = W.orderBy("unique_account_id", "date","frontend_id","network_id","frontend_name", "parent_frontend")
join_df = join_df.withColumn("TransactionSeq",  F.row_number().over(windowSpec)) \
                 .withColumn("EventTimestamp", lit(timestamp))

# Choose the columns in specific order
final_trans_df = join_df.select(join_df.TransactionSeq.cast(IntegerType()).alias('TransactionSeq'), \
                                from_unixtime(unix_timestamp('date', 'yyyy-MM-dd HH:mm:ss'), 'dd-MMM-yyyy HH:mm:ss').alias('date'), \
                                "frontend_id", "network_id", "frontend_name", "parent_frontend", \
                                "unique_account_id", "affiliate_id", "alias", \
                                userTransactionType_df["FinalCategory"],
                                "transaction_type", "sub_transaction_type", userTransactionType_df["Source1"], \
                                userTransactionType_df["Source2"], \
                                userTransactionType_df["Source3"], \
                                userTransactionType_df["Source4"], "status", "is_manual", "device_type", \
                                "operating_system", "currency", join_df.amount.cast(FloatType()).alias('amount'), \
                                join_df.TransactionId.cast(IntegerType()).alias('TransactionId'), \
                                "note", "EventTimestamp")
final_trans_df.show()

final_trans_df.write.mode("overwrite").saveAsTable("userTransData")

# 1.Format fields to right Datatypes and Cleanse them (special char), Date in DD-Mon-YYYY HH24:MI:SS
# 2.Link key from table   with fileds transaction_type,sub_transaction_type,source1,source2,source3,source4

+--------------+----+-----------+----------+-------------+---------------+-----------------+------------+-----+-------------+----------------+--------------------+-------+-------+-------+-------+------+---------+-----------+----------------+--------+------+-------------+----+--------------+
TransactionSeq|date|frontend_id|network_id|frontend_name|parent_frontend|unique_account_id|affiliate_id|alias|FinalCategory|transaction_type|sub_transaction_type|Source1|Source2|Source3|Source4|status|is_manual|device_type|operating_system|currency|amount|TransactionId|note|EventTimestamp|
+--------------+----+-----------+----------+-------------+---------------+-----------------+------------+-----+-------------+----------------+--------------------+-------+-------+-------+-------+------+---------+-----------+----------------+--------+------+-------------+----+--------------+
+--------------+----+-----------+----------+-------------+---------------+-----------------+------------+-----+-------------+----------------+--------------------+-------+-------+-------+-------+------+---------+-----------+----------------+--------+------+-------------+----+--------------+

Out[ 9 ]: 'None'